Note: if you don't have astropy yet, download and install astropy from this website: https://www.astropy.org/

In [1]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
from astropy.table import Column
import numpy as np

In [2]:
# download this from the same repo where you download this notebook 
from search_mast_cos_stis import search_mast_cos_stis

In [3]:


# you should be able to find most of the codes here: 
# https://github.com/yzhenggit/yztools

from yztools.mstar2mhalo import mstar2mhalo # return mhalo 
from yztools.calc_r200 import calc_r200  # input mahlo, return r200c, r200m 
from yztools.kpc2deg import kpc2deg # input impact_kpc, distance_kpc
from yztools.deg2kpc import deg2kpc # input impact_kpc, distance_kpc

# If you only have one target 

Input the target's coordinate, you need the target's ra/dec (or glon/glat) and distance 

In [4]:
tar_name = 'Sculptor Group (3.9Mpc)'
# tar_ra = '14:44:13' 
# tar_dec = '42:37:44'  
# tar_coord = SkyCoord(ra=tar_ra, dec=tar_dec, unit=(u.hour, u.deg), frame='icrs') 

tar_gl = 358.3 # deg
tar_gb = -77.3 # eye ball from Putman+2003
tar_coord = SkyCoord(l=tar_gl, b=tar_gb, unit=(u.deg, u.deg), frame='galactic') 

tar_dist_kpc = 3.9*1e3 # kpc 

print(tar_coord.icrs.ra.hms, tar_coord.icrs.dec.dms)

tar_ra_deg = tar_coord.icrs.ra.deg
tar_dec_deg = tar_coord.icrs.dec.deg

hms_tuple(h=0.0, m=1.0, s=9.389743290111392) dms_tuple(d=-33.0, m=-46.0, s=-52.159590073602544)


Now mstar, mhalo, and r200 if possible. Not Necessary to run this part if you have search_r_kpc in mind already, couldjust set a search radius (search_r_kpc) by hand in the next section.

In [5]:
try: 
    log_Mstar = 12.1
    log_Mhalo = np.log10(mstar2mhalo(10**log_Mstar))
    r200c, r200m = calc_r200(10**log_Mhalo)
    r200c = r200c.value
    r200m = r200m.value
    print(r200c, r200m)
except: 
    r200c = 400 
    r200m = 400

if r200m > 400: 
    r200c = 400 # try a random value 
    r200m = 400

In [6]:
print(r200c)

400


Convert r200 (let's use r200m) from kpc to deg at the corresponding distance 

Now search in MAST

In [7]:
search_r_kpc = 1000 # kpc # r200m

In [6]:
res = search_mast_cos_stis(tar_name, tar_ra_deg, tar_dec_deg,
                           tar_dist_kpc, filters=['G130M', 'G160M'],
                           search_r_kpc=search_r_kpc, yes_print=True)  

ok, input instrument_name = COS/FUV
**************************************************
Searching MAST within 14.69 deg/1000.0 kpc of Sculptor Group (3.9Mpc) 
Sculptor Group (3.9Mpc) at RA=0.2891, DEC=-33.7812, l=358.3000, b=-77.3000
Found 146 entries, save mast search result to ./mast_search_results/MAST_COS-FUV_Sculptor Group (3.9Mpc)_14.7deg_1000.0kpc_2021Jun22.csv


--------------- MAST/COS/FUV Search ---------------
Found: 2MASSJ00393579-3816584
impact=610.3 kpc, ra=9.8998, dec=-38.2833, l=314.7357, b=-78.5723
Observation Details: 
    Filter     ExpTime         PI     ID
     G130M       1259s   Shkolnik  14784
     G160M       1523s   Shkolnik  14784


--------------- MAST/COS/FUV Search ---------------
Found: CTS487
impact=551.6 kpc, ra=350.5457, dec=-34.7992, l=5.5446, b=-69.4434
Observation Details: 
    Filter     ExpTime         PI     ID
     G160M          5s        Fox  13448
     G160M        318s        Fox  13448
     G160M        104s        Fox  13448
     G130M     

     G130M       4407s     Wakker  13444
     G130M        648s     Wakker  13444
     G130M        133s     Wakker  13444


--------------- MAST/COS/FUV Search ---------------
Found: SB357
impact=741.4 kpc, ra=13.2564, dec=-36.3387, l=300.9353, b=-80.7834
Observation Details: 
    Filter     ExpTime         PI     ID
     G130M          0s     Lehner  11592
     G130M        382s     Lehner  11592
     G130M        112s     Lehner  11592
     G130M        253s     Lehner  11592
     G130M        253s     Lehner  11592
     G160M        348s     Lehner  11592
     G160M        363s     Lehner  11592


--------------- MAST/COS/FUV Search ---------------
Found: VV2006-J001641.9-312657
impact=273.4 kpc, ra=4.1745, dec=-31.4491, l=1.1544, b=-81.2841
Observation Details: 
    Filter     ExpTime         PI     ID
     G130M       7899s     Putman  16301
     G160M       7624s     Putman  16301
     G160M       5216s     Putman  16301


--------------- MAST/COS/FUV Search ---------------
Foun

In [15]:
res

# If you have a table of targets (to be added, 01/25/2021)

First read in a table using astropy 

In [13]:
tar_table = Table.read('tables/dwarfs_info.txt', format='ascii')
tar_table.show_in_notebook()

idx,Name,RA(hms),Dec(dms),vlsr(kms),Dist(kpc),Mstar,PI
0,LGS3,01:03:55.0,+21:53:06,–286.5,769,960000.0,MP
1,ESO410-G005,00:15:31.6,-32:10:48,159,1923,3500000.0,MP
2,WLM,00:01:58.2,-15:27:39,–130,933,43000000.0,MP
3,DDO190,14:24:43.4,+44:31:33,150,2793,51000000.0,MP
4,IC1613,01:04:47.8,+02:07:4,-237.4,755,100000000.0,YZ
5,Sextans-B,10:00:0.8,+05:19:56,304,1426,52000000.0,JB
6,Sextans-A,10:11:0.8,-04:41:34,324,1432,44000000.0,JB
7,GR8,12:58:40.4,+14:13:03,213.9,2178,6400000.0,JB
8,NGC3109,10:03:6.9,-26:09:35,403,1300,76000000.0,JB
9,NGC4214,12:15:39.174,+36:19:36.80,291,2880,590000000.0,JB


Calculate the dark matter halo mass and virial radii (R200) for these dwarfs using their stellar masses. And integrate the result into the dw_table 

In [17]:
mhalo_all = []
r200_all = []

for iw in range(len(dw_table)): 
    dw_name = dw_table['Name'][iw]
    mstar = dw_table['Mstar'][iw]
    mhalo, r200 = mhalo_r200(mstar) 
    # print('%10s, mhalo=%.1e Msun, R200=%.2f kpc'%(dw_name, mhalo, r200))
    mhalo_all.append(np.around(np.log10(mhalo), decimals=2))
    r200_all.append(np.around(r200, decimals=1))
    
r200_col = Column(np.asarray(r200_all), name='R200(kpc)')
mhalo_col = Column(np.asarray(mhalo_all), name='logMh')
dw_all = dw_table.copy()
dw_all.add_column(mhalo_col, index=7)
dw_all.add_column(r200_col, index=8)
dw_all.show_in_notebook()

idx,Name,RA(hms),Dec(dms),vlsr(kms),Dist(kpc),Mstar,PI,logMh,R200(kpc)
0,LGS3,01:03:55.0,+21:53:06,–286.5,769,960000.0,MP,9.63,50.5
1,ESO410-G005,00:15:31.6,-32:10:48,159,1923,3500000.0,MP,9.9,62.2
2,WLM,00:01:58.2,-15:27:39,–130,933,43000000.0,MP,10.43,93.5
3,DDO190,14:24:43.4,+44:31:33,150,2793,51000000.0,MP,10.46,96.1
4,IC1613,01:04:47.8,+02:07:4,-237.4,755,100000000.0,YZ,10.61,107.2
5,Sextans-B,10:00:0.8,+05:19:56,304,1426,52000000.0,JB,10.47,96.4
6,Sextans-A,10:11:0.8,-04:41:34,324,1432,44000000.0,JB,10.43,93.8
7,GR8,12:58:40.4,+14:13:03,213.9,2178,6400000.0,JB,10.03,68.6
8,NGC3109,10:03:6.9,-26:09:35,403,1300,76000000.0,JB,10.55,102.5
9,NGC4214,12:15:39.174,+36:19:36.80,291,2880,590000000.0,JB,10.99,143.5


Now find QSO sightlines within 1 virial radius (R200) of the galaxies

In [19]:
for iw in range(len(dw_all)): 
    dw_name = dw_all['Name'][iw] # or other galaxies of your choice 
    dw_ra = dw_all['RA(hms)'][iw]      # coordinate of the galaxy 
    dw_dec = dw_all['Dec(dms)'][iw]
    dw_dist_kpc = dw_all['Dist(kpc)'][iw]   # distance to the galaxy 
    search_r_kpc = dw_all['R200(kpc)'][iw]  # search within this radius of the galaxy
    
    ### Hamilton, if your ra and dec are in degrees, use this line instead 
    # dw_coord = SkyCoord(ra=dw_ra, dec=dw_dec, unit=(u.deg, u.deg), frame='icrs')
    dw_coord = SkyCoord(ra=dw_ra, dec=dw_dec, unit=(u.hour, u.deg), frame='icrs')
    input_ra = dw_coord.icrs.ra.deg
    input_dec = dw_coord.icrs.dec.deg
    
    # use this try/except to catch errors 
    try: 
        res = search_mast_cos_stis(dw_name, input_ra, input_dec,
                               dw_dist_kpc, filters=['G130M', 'G160M'],
                               search_r_kpc=search_r_kpc, yes_print=True)  
    except: 
        print("********* Codes fail when running for %s ********** "%(dw_name))

ok, input instrument_name = COS/FUV
**************************************************
Searching MAST within 3.76 deg/50.5 kpc of LGS3 
LGS3 at RA=15.9792, DEC=21.8850, l=126.7625, b=-40.8939
Found 8 entries, save mast search result to ./MAST_COS-FUV_LGS3_3.8deg_50.5kpc_2020Jul02.csv


--------------- MAST/COS/FUV Search ---------------
Found: J010534+234960
impact=26.6 kpc, ra=16.3906, dec=23.8333, l=127.0847, b=-38.9245
Observation Details: 
    Filter     ExpTime         PI     ID
     G160M       2472s      Henry  15865


--------------- MAST/COS/FUV Search ---------------
Found: RXSJ00537+2232
impact=32.7 kpc, ra=13.4423, dec=22.5395, l=123.6381, b=-40.3291
Observation Details: 
    Filter     ExpTime         PI     ID
     G130M          5s     Lehner  14268
     G130M        436s     Lehner  14268
     G160M       2249s     Lehner  14268
     G130M       1700s     Lehner  14268
     G160M       2228s     Lehner  14268
     G130M        106s     Lehner  14268
     G130M       204

Sorry, Find nothing.
ok, input instrument_name = COS/FUV
**************************************************
Searching MAST within 5.74 deg/93.5 kpc of WLM 
WLM at RA=0.4925, DEC=-15.4608, l=75.8637, b=-73.6244
Found 35 entries, save mast search result to ./MAST_COS-FUV_WLM_5.7deg_93.5kpc_2020Jul02.csv


--------------- MAST/COS/FUV Search ---------------
Found: PHL2525
impact=44.4 kpc, ra=0.1018, dec=-12.7633, l=80.6835, b=-71.3172
Observation Details: 
    Filter     ExpTime         PI     ID
     G130M          3s        Fox  12604
     G130M       2146s        Fox  12604
     G160M       2772s        Fox  12604
     G130M        107s        Fox  12604
     G130M        256s        Fox  12604


--------------- MAST/COS/FUV Search ---------------
Found: PMNJ2345-1555
impact=66.1 kpc, ra=356.3019, dec=-15.9188, l=65.6728, b=-70.9851
Observation Details: 
    Filter     ExpTime         PI     ID
     G160M       2560s     Stocke  13008
     G160M       1260s     Stocke  13008
     G130M

     G160M         13s       Lanz  12867
     G160M        123s       Lanz  12867
     G160M        551s       Lanz  12867


--------------- MAST/COS/FUV Search ---------------
Found: IC1613-B11
impact=0.2 kpc, ra=16.1825, dec=2.1124, l=129.7052, b=-60.5844
Observation Details: 
    Filter     ExpTime         PI     ID
     G160M        131s       Lanz  12867
     G130M       2729s       Lanz  12867
     G130M         11s       Lanz  12867
     G160M         18s       Lanz  12867
     G130M        477s       Lanz  12867
     G130M        113s       Lanz  12867
     G160M       2729s       Lanz  12867
     G160M       2729s       Lanz  12867
     G160M        596s       Lanz  12867
     G160M       2729s       Lanz  12867
     G130M       4210s Roman-Duval  15880
     G130M       1852s       Lanz  12867
     G160M       1740s       Lanz  12867


--------------- MAST/COS/FUV Search ---------------
Found: IC1613-B2
impact=1.1 kpc, ra=16.2628, dec=2.1679, l=129.8555, b=-60.5207
Observation

Found 50 entries, save mast search result to ./MAST_COS-FUV_Sextans-A_3.8deg_93.8kpc_2020Jul02.csv


--------------- MAST/COS/FUV Search ---------------
Found: MRK-1253
impact=63.1 kpc, ra=154.8869, dec=-3.3375, l=246.6971, b=42.3646
Observation Details: 
    Filter     ExpTime         PI     ID
     G130M        840s    Bregman  13347
     G130M        675s    Bregman  13347
     G130M        825s    Bregman  13347
     G130M        825s    Bregman  13347
     G160M       1074s    Bregman  13347
     G160M       1064s    Bregman  13347
     G160M        850s    Bregman  13347
     G160M        850s    Bregman  13347


--------------- MAST/COS/FUV Search ---------------
Found: PG1011-040
impact=22.8 kpc, ra=153.5862, dec=-4.3112, l=246.5011, b=40.7488
Observation Details: 
    Filter     ExpTime         PI     ID
     G160M        112s      Green  11524
     G130M       1307s      Green  11524
     G160M       1381s      Green  11524
     G160M       1381s      Green  11524
     G160M 

Found 15 entries, save mast search result to ./MAST_COS-FUV_NGC4214_2.9deg_143.5kpc_2020Jul02.csv


--------------- MAST/COS/FUV Search ---------------
Found: MRK646
impact=115.6 kpc, ra=181.4571, dec=35.1794, l=170.5586, b=77.3657
Observation Details: 
    Filter     ExpTime         PI     ID
     G130M       2158s     Wakker  14772


--------------- MAST/COS/FUV Search ---------------
Found: NGC4214-OBJECT-1
impact=0.1 kpc, ra=183.9145, dec=36.3265, l=160.2519, b=78.0755
Observation Details: 
    Filter     ExpTime         PI     ID
     G160M       1156s     Aloisi  15193
     G130M        540s     Aloisi  15193
     G130M       1968s     Aloisi  11579


--------------- MAST/COS/FUV Search ---------------
Found: NGC4214-OBJECT-1-REVISIT
impact=0.1 kpc, ra=183.9145, dec=36.3265, l=160.2519, b=78.0755
Observation Details: 
    Filter     ExpTime         PI     ID
     G130M       1968s     Aloisi  11579


--------------- MAST/COS/FUV Search ---------------
Found: SDSSJ121114.56+365739

In [7]:
for iw in range(3):
    coord = SkyCoord(ra=dw['RA(hms)'][iw], dec=dw['Dec(dms)'][iw], unit=(u.hourangle, u.degree))
    search_mast_qso_cosfuv(dw['Name'][iw], coord.icrs.ra.degree, coord.icrs.dec.degree, 
                           gal_dist_kpc=dw['Dist(kpc)'][iw], within_radius_kpc = dw['Rvir(kpc)'][iw]*1.5)

********************************************************************************
Searching MAST within 51.0 kpc of LGS3 (RA=15.9792, DEC=21.8850, l=126.7625, b=-40.8939)
8
--------------- MAST Archival Search ---------------
Found: IC-1586
impact=50.1 kpc, ra=11.9856, dec=22.3724, l=121.8693, b=-40.4922
Observation Details: 
    Filter     ExpTime         PI     ID
     G140L        720s   Levesque  13481


--------------- MAST Archival Search ---------------
Found: RXSJ00537+2232
impact=32.7 kpc, ra=13.4415, dec=22.5397, l=123.6371, b=-40.3289
Observation Details: 
    Filter     ExpTime         PI     ID
     G130M       1700s     Lehner  14268
     G130M          5s     Lehner  14268
     G160M       2249s     Lehner  14268
     G130M       2049s     Lehner  14268
     G130M        106s     Lehner  14268
     G130M        435s     Lehner  14268
     G160M       2228s     Lehner  14268


********************************************************************************
Searching MAST w